In [ ]:
import psycopg2
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import yaml
import matplotlib.pyplot as plt
from plotnine import *

file = open("../pop.yaml", 'r')
config = yaml.load(file)

engine_path = 'postgresql://{}:{}@{}/{}'
engine = create_engine(engine_path.format(config['db']['username'], config['db']['password'], config['db']['host'],
                                          config['db']['database']))

In [ ]:
#importing required packages -- bokeh, geopandas, shapely, fiona
import requests
import zipfile
import io 


!pip install fiona
import fiona

import geopandas as gp

!pip install shapely 
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

!pip install pysal
import pysal as ps

import matplotlib.pyplot as plt

from bokeh.io import show, output_file, output_notebook, export_png
from bokeh.models import ColumnDataSource, HoverTool, PanTool, WheelZoomTool, LinearColorMapper, BasicTicker, ColorBar, LogColorMapper
from bokeh.palettes import brewer 
from bokeh.plotting import figure, save
from bokeh.resources import CDN
from bokeh.transform import transform
output_notebook()

In [ ]:
#Region wise chile map
url = 'https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10398/1/division_regional.zip'
request_file = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_file.content))
file_zip.extractall()

In [ ]:
#reading the shapefile with geopandas
regions = gp.read_file('division_regional.shp')

In [ ]:
p = regions.plot()

In [ ]:
#splitting multipolygons into single polygons for efficient extraction of lat long/ x y co-ords
extract_multi = regions.set_index(['NOM_REG'])['geometry'].apply(pd.Series).stack().reset_index()

In [ ]:
#fucntion for extraction of x,y co-ords
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [ ]:
extract_multi['x'] = extract_multi.apply(getPolyCoords, geom=0, coord_type='x', axis=1)
extract_multi['y'] = extract_multi.apply(getPolyCoords, geom=0, coord_type='y', axis=1)

In [ ]:
def region_to_number(var):
    if(var == "Región de Valparaíso"): 
        return('5')
    elif(var == "Región del Libertador Bernardo O'Higgins"): 
        return('6')
    elif(var == "Región de Los Lagos"): 
        return('10')
    elif(var == "Región de Atacama"): 
        return('3')
    elif(var == "Región Metropolitana de Santiago"): 
        return('13')
    elif(var == "Región de Aysén del Gral.Ibañez del Campo"): 
        return('11')
    elif(var == "Región de La Araucanía"): 
        return('9')
    elif(var == "Región del Maule"): 
        return('7')
    elif(var == "Región de Tarapacá"): 
        return('1')
    elif(var == "Región de Los Ríos"): 
        return('14')
    elif(var == "Región de Arica y Parinacota"): 
        return('15')
    elif(var == "Región de Antofagasta"): 
        return('2')
    elif(var == "Región de Coquimbo"): 
        return('4')
    elif(var == "Región del Bío-Bío"): 
        return('8')
    else: #(NOM_REG == "Región de Magallanes y Antártica Chilena"): 
        return('12')

In [ ]:
extract_multi['Region_Code'] = extract_multi.NOM_REG.apply(region_to_number)

In [ ]:
#preparing df to build map on
qry = """
        set role direccion_trabajo_inspections_write;
        SELECT count(IDFiscalizacion) as No_Inspections, Region
        FROM raw.inspections_complete
        group by Region;
        """

inspections_per_region = pd.read_sql_query(qry, engine)

inspections_per_region = inspections_per_region.rename(columns={'region': 'Region_Code'})

In [ ]:
#merging on region number
extract_multi = extract_multi.merge(inspections_per_region, 'left', on='Region_Code')

In [ ]:
from bokeh.models import NumeralTickFormatter
from bokeh.models import PrintfTickFormatter


In [ ]:
#make map
colors = brewer['RdPu'][9]
mapper = LinearColorMapper(palette=colors,low=extract_multi.no_inspections.min(), 
                           high=extract_multi.no_inspections.max(), nan_color='#EAE8E7')

source = ColumnDataSource(data=dict(
    x=extract_multi['x'], 
    y=extract_multi['y'], 
    inspections=extract_multi['no_inspections'],
    reg_name=extract_multi['NOM_REG']
))

p = figure(
    title="No. of Inspections by Region in Chile",
    x_axis_location=None, y_axis_location=None,
    plot_width=2500, plot_height=900
)

p.patches('x', 'y', source=source,
          fill_color=transform('inspections', mapper),
          fill_alpha=0.8, line_color="black", line_width=0.5)

color_bar = ColorBar(color_mapper=mapper, ticker=BasicTicker(),
                 label_standoff=13, border_line_color=None, location=(0,0),
                 major_label_text_font_size='13pt'
                 major_label_text_font = 'nunito')

p.add_layout(color_bar, 'right')

#Add tools
hover= HoverTool(tooltips = [
   ("Region Name","@reg_name"),
    ("No. of Inspections","@inspections")])

p.add_tools(PanTool(), WheelZoomTool(), hover)


In [ ]:
#output_file("test.html")
#save(p)

In [ ]:
show(p)

In [ ]:
#comuna map
url = 'https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10396/1/division_comunal.zip'
request_file = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_file.content))
file_zip.extractall()

In [ ]:
comunas = gp.read_file('division_comunal.shp')

In [ ]:
#c = comunas.plot()

In [ ]:
comunas.head(5)

In [ ]:
qry = """
        set role direccion_trabajo_inspections_write;
        SELECT count(IDFiscalizacion) as No_Inspections, empdfcodcomuna
        FROM raw.inspections_complete
        group by empdfcodcomuna;
        """

In [ ]:
comuna_inspections =pd.read_sql_query(qry, engine)

In [ ]:
comuna_inspections = comuna_inspections.rename(columns={'empdfcodcomuna': 'COD_COMUNA'})

In [ ]:
comuna_inspections["COD_COMUNA"] = comuna_inspections.COD_COMUNA.astype(float)

In [ ]:
comuna_mp = comuna.set_index(['COD_COMUNA'])['geometry'].apply(pd.Series).stack().reset_index()

In [ ]:
comuna_mp["COD_COMUNA"] = comuna_mp.COD_COMUNA.astype(float)

In [ ]:
comunas_mp['x'] = comunas_mp.apply(getPolyCoords, geom=0, coord_type='x', axis=1)
comunas_mp['y'] = comunas_mp.apply(getPolyCoords, geom=0, coord_type='y', axis=1)

In [ ]:
comunas_mp = comunas_mp.merge(comuna_inspections, 'left', on='COD_COMUNA')

In [ ]:
#make map
colors = brewer['RdPu'][9]
mapper = LinearColorMapper(palette=colors,low=extract_multi.no_inspections.min(), 
                           high=extract_multi.no_inspections.max(), nan_color='#EAE8E7')

source = ColumnDataSource(data=dict(
    x=extract_multi['x'], 
    y=extract_multi['y'], 
    inspections=extract_multi['no_inspections']
))

c = figure(
    title="Heatmap for No. of Inspections/ Region in Chile",
    x_axis_location=None, y_axis_location=None,
    plot_width=2500, plot_height=900
)

c.patches('x', 'y', source=source,
          fill_color=transform('inspections', mapper),
          fill_alpha=0.8, line_color="black", line_width=0.5)

color_bar = ColorBar(color_mapper=mapper, ticker=BasicTicker(),
                 label_standoff=12, border_line_color=None, location=(0,0),
                 major_label_text_font = 'No. of Inspections')

c.add_layout(color_bar, 'right')

In [ ]:
show(c)